# Практика

In [42]:
from analytics_lib import AnyName

In [43]:
AnyName().useful_func(1, 2, 3)

Выполняется при обращении к классу


[1, 4, 9]

In [ ]:
from analytics_lib import *  # может быть конфликт имен

In [ ]:
def useful_func(*args):
    return [x**2 for x in args]

In [8]:
class AnyName:
    def __init__(self, currency='GBP', *args, **kwargs):
        # Создаем все атрибуты, которые нам понадобятся
        print('Выполняется при обращении к классу')
        self.currency = currency
        self.value = 100
        
        # приватные атрибуты
        self._api_version = 'v1'  # намек, что менять этот атрибут извне не следует
        self.__private_attr = 123  # запрет на изменение
    
    # функции внутри классов называются методами
    def method_1(self):
        # currency = 'RUB'  # currency - просто переменная
        self.currency = 'RUB'  # это атрибут
        
    def method_2(self):
        print(self.currency, self.__private_attr)
        
    @staticmethod
    def useful_func(*args):
        return [x**2 for x in args]
    
    @property
    def value_with_vat(self):
        return str(self.value * 1.2) + self.currency

In [9]:
a = AnyName()

Выполняется при обращении к классу


In [10]:
a.value_with_vat

'120.0GBP'

In [41]:
a.useful_func(1, 2, 3)

[1, 4, 9]

In [38]:
a.method_2()

GBP 123


In [35]:
a.__private_attr

AttributeError: 'AnyName' object has no attribute '__private_attr'

In [32]:
a._api_version = 'v2'
a._api_version

'v2'

In [28]:
a.currency

'GBP'

In [22]:
a.method_2()

USD


In [13]:
AnyName().method_1()

In [9]:
a.method_1()

In [11]:
a.currency = 'EUR'

In [12]:
a.method_2()

EUR


In [15]:
1 + 2

3

In [16]:
'1' + '2'

'12'

In [14]:
dir(a)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'currency',
 'method_1',
 'method_2']

In [4]:
def func_1():
    currency = 'RUB'
    return currency
    
    
def func_2(currency):
    print(currency)

In [5]:
result = func_1()

In [6]:
func_2(result)

RUB


# Требования к отчету
(в хронологическом порядке)

1. Нужен отчет по трафику (визиты по дням)
2. Сделайте таблицу попроще
3. А можно сразу очень много строк получить?

[Демо-счетчик](https://metrika.yandex.ru/stat/traffic?sort=-ym%3As%3Avisits&period=2020-08-01%3A2020-09-06&accuracy=1&id=44147844&stateHash=5d3ee51607d6bde783131e11) Яндекс.Метрики


In [11]:
import requests

In [29]:
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
METRIKA_ROWS_LIMIT = 5  # max 100000

DEMO_COUNTER = 44147844

In [30]:
class Metrika:
    def __init__(self, token='', counter=DEMO_COUNTER, start_date='2020-09-01', end_date='2020-09-10'):
        self.counter = counter
        self.limit = METRIKA_ROWS_LIMIT
        self.start_date = start_date
        self.end_date = end_date
        
        if token:
            self.headers = {'Authorization': 'OAuth ' + token}
        else:
            self.headers = ''  # для демо-счетчика заголовок запроса должен быть пустым
        
        self.dimensions = ''  # строки отчета
        self.metrics = ''  # столбцы отчета
        self.report = []  # собственно сам отчет
        
    def api_request(self, offset=1):
        """Запрос к API Метрики, возвращает словарь с отчетом"""
        params = {
            'id': self.counter,
            'date1': self.start_date,
            'date2': self.end_date,
            'metrics': self.metrics,
            'dimensions': self.dimensions,
            'limit': self.limit,
            'offset': offset,
            'accuracy': 1,
        }
        
        response = requests.get(API_URL, params=params, headers=self.headers)
        return response.json()['data']
    
    def traffic(self):
        """Отчет по посещаемости"""
        self.metrics = 'ym:s:visits'  # как правило эти атрибуты будут в init
        self.dimensions = 'ym:s:date'
        
        # report = self.api_request()
        report = self.full_report()
        self.report = report
        
    def browsers(self):
        """Отчет по браузерам"""
        self.metrics = 'ym:s:visits'
        self.dimensions = 'ym:s:date,ym:s:browser'
        
        # report = self.api_request()
        report = self.full_report()
        self.report = report
    
    def reformat_api_report(self):
        """Упрощение отчета до списка списков"""
        reformatted_report = []

        for line in self.report:
            dimensions = [x['name'] for x in line['dimensions']]
            reformatted_report.append(dimensions + line['metrics'])

        self.report_formatted = reformatted_report
    
    def full_report(self):
        """Постраничная выгрузка из Метрики"""
        full_data = []
        offset = 1
        
        while True:
            print('Starting offset {}'.format(offset))
            data = self.api_request(offset=offset)
            full_data += data
            
            offset += self.limit
            if not data or len(data) < self.limit:
                break
        
        return full_data

In [31]:
m = Metrika(start_date='2023-01-01', end_date='2023-02-20')

In [32]:
m.traffic()

Starting offset 1
Starting offset 6
Starting offset 11
Starting offset 16
Starting offset 21
Starting offset 26
Starting offset 31
Starting offset 36
Starting offset 41
Starting offset 46
Starting offset 51


In [33]:
m.report

[{'dimensions': [{'name': '2023-01-19'}], 'metrics': [921.0]},
 {'dimensions': [{'name': '2023-01-11'}], 'metrics': [910.0]},
 {'dimensions': [{'name': '2023-01-30'}], 'metrics': [860.0]},
 {'dimensions': [{'name': '2023-01-12'}], 'metrics': [839.0]},
 {'dimensions': [{'name': '2023-01-10'}], 'metrics': [836.0]},
 {'dimensions': [{'name': '2023-01-18'}], 'metrics': [834.0]},
 {'dimensions': [{'name': '2023-01-16'}], 'metrics': [826.0]},
 {'dimensions': [{'name': '2023-01-17'}], 'metrics': [823.0]},
 {'dimensions': [{'name': '2023-01-26'}], 'metrics': [810.0]},
 {'dimensions': [{'name': '2023-01-09'}], 'metrics': [797.0]},
 {'dimensions': [{'name': '2023-01-13'}], 'metrics': [767.0]},
 {'dimensions': [{'name': '2023-01-27'}], 'metrics': [726.0]},
 {'dimensions': [{'name': '2023-01-31'}], 'metrics': [713.0]},
 {'dimensions': [{'name': '2023-01-25'}], 'metrics': [665.0]},
 {'dimensions': [{'name': '2023-01-20'}], 'metrics': [631.0]},
 {'dimensions': [{'name': '2023-01-24'}], 'metrics': [6

In [34]:
m.reformat_api_report()

In [35]:
m.report_formatted

[['2023-01-19', 921.0],
 ['2023-01-11', 910.0],
 ['2023-01-30', 860.0],
 ['2023-01-12', 839.0],
 ['2023-01-10', 836.0],
 ['2023-01-18', 834.0],
 ['2023-01-16', 826.0],
 ['2023-01-17', 823.0],
 ['2023-01-26', 810.0],
 ['2023-01-09', 797.0],
 ['2023-01-13', 767.0],
 ['2023-01-27', 726.0],
 ['2023-01-31', 713.0],
 ['2023-01-25', 665.0],
 ['2023-01-20', 631.0],
 ['2023-01-24', 617.0],
 ['2023-02-02', 617.0],
 ['2023-01-23', 603.0],
 ['2023-02-01', 595.0],
 ['2023-02-06', 583.0],
 ['2023-02-07', 565.0],
 ['2023-02-08', 561.0],
 ['2023-02-03', 560.0],
 ['2023-02-15', 548.0],
 ['2023-02-20', 536.0],
 ['2023-01-05', 521.0],
 ['2023-02-09', 521.0],
 ['2023-02-16', 521.0],
 ['2023-01-04', 512.0],
 ['2023-02-14', 499.0],
 ['2023-02-13', 497.0],
 ['2023-01-15', 488.0],
 ['2023-01-03', 476.0],
 ['2023-02-17', 452.0],
 ['2023-01-29', 451.0],
 ['2023-02-10', 447.0],
 ['2023-01-14', 439.0],
 ['2023-01-06', 433.0],
 ['2023-01-07', 432.0],
 ['2023-01-08', 418.0],
 ['2023-01-28', 396.0],
 ['2023-01-02', 

In [36]:
m.browsers()
m.report

Starting offset 1
Starting offset 6
Starting offset 11
Starting offset 16
Starting offset 21
Starting offset 26
Starting offset 31
Starting offset 36
Starting offset 41
Starting offset 46
Starting offset 51
Starting offset 56
Starting offset 61
Starting offset 66
Starting offset 71
Starting offset 76
Starting offset 81
Starting offset 86
Starting offset 91
Starting offset 96
Starting offset 101
Starting offset 106
Starting offset 111
Starting offset 116
Starting offset 121
Starting offset 126
Starting offset 131
Starting offset 136
Starting offset 141
Starting offset 146
Starting offset 151
Starting offset 156
Starting offset 161
Starting offset 166
Starting offset 171
Starting offset 176
Starting offset 181
Starting offset 186
Starting offset 191
Starting offset 196
Starting offset 201
Starting offset 206
Starting offset 211
Starting offset 216
Starting offset 221
Starting offset 226
Starting offset 231
Starting offset 236
Starting offset 241
Starting offset 246
Starting offset 251
St

[{'dimensions': [{'name': '2023-01-30'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [408.0]},
 {'dimensions': [{'name': '2023-01-11'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [374.0]},
 {'dimensions': [{'name': '2023-01-19'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [361.0]},
 {'dimensions': [{'name': '2023-01-26'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [345.0]},
 {'dimensions': [{'name': '2023-01-12'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [340.0]},
 {'dimensions': [{'name': '2023-01-17'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [331.0]},
 {'dimensions': [{'name': '2023-01-18'},
   {'icon_i

In [37]:
m.reformat_api_report()
m.report_formatted

[['2023-01-30', 'Google Chrome', 408.0],
 ['2023-01-11', 'Google Chrome', 374.0],
 ['2023-01-19', 'Google Chrome', 361.0],
 ['2023-01-26', 'Google Chrome', 345.0],
 ['2023-01-12', 'Google Chrome', 340.0],
 ['2023-01-17', 'Google Chrome', 331.0],
 ['2023-01-18', 'Google Chrome', 330.0],
 ['2023-01-16', 'Google Chrome', 325.0],
 ['2023-01-31', 'Google Chrome', 323.0],
 ['2023-01-09', 'Google Chrome', 320.0],
 ['2023-01-10', 'Google Chrome', 319.0],
 ['2023-01-13', 'Google Chrome', 304.0],
 ['2023-01-27', 'Google Chrome', 300.0],
 ['2023-01-25', 'Google Chrome', 273.0],
 ['2023-02-01', 'Google Chrome', 258.0],
 ['2023-02-02', 'Google Chrome', 256.0],
 ['2023-02-06', 'Google Chrome', 243.0],
 ['2023-02-20', 'Google Chrome', 243.0],
 ['2023-02-15', 'Google Chrome', 240.0],
 ['2023-01-20', 'Google Chrome', 237.0],
 ['2023-02-07', 'Google Chrome', 235.0],
 ['2023-02-08', 'Google Chrome', 235.0],
 ['2023-02-16', 'Google Chrome', 230.0],
 ['2023-02-03', 'Google Chrome', 219.0],
 ['2023-01-24', 